In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#@title Enter Path with PDF Papers
path = "" # @param {type:"string"}

In [5]:
%cd $path

/content/drive/MyDrive/Digital Twin Project/Article Checker/Final


In [ ]:
%pip install PyPDF2 openai typing_extensions anthropic pytesseract pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.8/870.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
!sudo apt install tesseract-ocr -y
# Install Poppler
!apt-get install -y poppler-utils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (10.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [ ]:
!dir

Automate_GPT_10.ipynb  Automate_GPT_4.ipynb	     Checked
Automate_GPT_11.ipynb  Automate_GPT_5.ipynb	     conejos\ fuertes.pdf
Automate_GPT_12.ipynb  Automate_GPT_6.ipynb	     Corriveau\ et\ al.,\ 2024.pdf
Automate_GPT_13.ipynb  Automate_GPT_7.ipynb	     Lu\ et\ al\ 2019.pdf
Automate_GPT_14.ipynb  Automate_GPT_8.ipynb	     Pesantez\ et\ al..pdf
Automate_GPT_15.ipynb  Automate_GPT_9.ipynb	     Tripathi\ et\ al\ 2021.pdf
Automate_GPT_1.ipynb   Automate_GPT.ipynb	     Valverde-Perez\ et\ al.,\ 2021.pdf
Automate_GPT_2.ipynb   Bernard\ 2024.pdf	     van\ Rooij\ et\ al\ 2022.pdf
Automate_GPT_3.ipynb   brahmbhatt\ et\ al\ 2023.pdf  Zhou\ et\ al\ 2024.pdf


***You only need to enter the API key for the model you want to use.***

In [ ]:
#@title OpenAI API Key
api_key = "" # @param {type:"string"}

In [ ]:
#@title Claude (Anthropic) API Key
claude_api_key = "" # @param {type:"string"}

The code below uses OpenAI's GPT models. To use Claude, edit lines 200 and 202 as specified.

In [ ]:
import os
from pathlib import Path
import PyPDF2
import openai
import pandas as pd
import shutil
import time
from openai import RateLimitError
import anthropic

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text

def extract_title_with_gpt(client, text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Extract the title from the following article text:"},
            {"role": "user", "content": text[:500]}
        ],
        temperature=0,
        max_tokens=60,
        top_p=1
    )
    title = response.choices[0].message.content.strip()
    return title

def extract_author_with_gpt(client, text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Extract the last name of the main author from the following article text followed by et al. (e.g. 'Sanan et al.'):"},
            {"role": "user", "content": text[:500]}
        ],
        temperature=0,
        max_tokens=60,
        top_p=1
    )
    author = response.choices[0].message.content.strip()
    return author

def extract_industry_with_gpt(client, text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Tell the the industry name that the article focuses on (e.g. 'Water', 'Energy', etc.). If there is none write 'General':"},
            {"role": "user", "content": text[:1500]}
        ],
        temperature=0,
        max_tokens=60,
        top_p=1
    )
    industry = response.choices[0].message.content.strip()
    return industry

def explain_parameter_presence(client, text, parameter, definition):
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": f"Explain whether the parameter '{parameter}' is mentioned/directly talked about in the following text and provide evidence from the text. If it does, briefly explain how (3-5 sentences with ~2 pieces of evidence); if it does not match, briefly explain why the paper does not focus on it (1 sentence). Note that '{parameter}' is defined as '{definition}'."},
            {"role": "user", "content": text[:50000]}
        ],
        temperature=0.7,
        max_tokens=250,
        top_p=1
    )
    explanation = response.choices[0].message.content.strip()
    return explanation

def explain_parameter_presence2(client, text, parameter, definition):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": f"Explain whether the parameter '{parameter}' is mentioned/directly talked about in the following text and provide evidence from the text. If it does, briefly explain how (3-5 sentences with ~2 pieces of evidence); if it does not match, briefly explain why the paper does not focus on it (1 sentence). Note that '{parameter}' is defined as '{definition}'."},
            {"role": "user", "content": text[:50000]} #For token limit
        ],
        temperature=0.7,
        max_tokens=250,
        top_p=1
    )
    explanation = response.choices[0].message.content.strip()
    return explanation

def explain_parameter_presence3(client, text, parameter, definition):
    prompt = f"Explain whether the parameter '{parameter}' is mentioned in the following text and provide evidence from the text. If it does, briefly explain how (3-5 sentences with ~2 pieces of evidence); if it does not match, briefly explain why the paper does not focus on it (1 sentence). Note that '{parameter}' is defined as '{definition}'."

    message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1024,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "text", "text": text}  # Ensure text fits within API limits
                ]
            }
        ]
    )
    explanation = message.content[0].text.strip()
    return explanation


def explain_parameter_presence4(client, text, parameter, definition):
    prompt = f"Explain whether the parameter '{parameter}' is mentioned in the following text and provide evidence from the text. If it does, briefly explain how (3-5 sentences with ~2 pieces of evidence); if it does not match, briefly explain why the paper does not focus on it (1 sentence). Note that '{parameter}' is defined as '{definition}'."

    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1024,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "text", "text": text}  # Ensure text fits within API limits
                ]
            }
        ]
    )
    explanation = message.content[0].text.strip()
    return explanation


#Parameters and their definitions
parameters = [
    "Physical Entity and Processes", "Virtual Entity", "Virtual Processes",
    "Physical Environment", "Virtual Environment", "Realization", "State",
    "Metrology", "Fidelity", "Parameters", "Twinning Rate",
    "Use Cases", "Perceived Benefits", "Data Ownership", "Scope",
    "P2V Connection", "V2P Connection"
]

definitions = [
   "Physical Entity or Process: An object such as a piece of equipment, a component, a system, or real-world processes within a system, which are being represented and simulated by a digital twin. A paper that talks about this will say anything about the physical system/process that the Digital Twin represents.",
   "Virtual Entity: A digital counterpart or replica of a physical entity or process forming the digital twin environment. It exactly mimics the behavior, characteristics, and attributes of its physical counterpart in real time. It is usually represented as a digital model or several connected models. Models can include ANN, CNN, GNN, GA, Monte Carlo. A paper that talks about this will say anything about the virtual Digital Twin.",
   "Virtual Process: The computational techniques employed within the virtual entity to shadow the physical entity and provide additional insights. It involves replicating the behavior, interactions, and outcomes of the physical process using computational models and algorithms. It can include optimization, predictive analytics, simulation and experimentation.",
   "Physical Environment: Refers to the actual physical surroundings or context in which a system operates, including physical surroundings, infrastructure, equipment, facilities, and natural elements.",
   "Virtual Environment: The various models, simulations, or processes that the virtual entity interacts with to digitally replicate the physical entity. This also includes any software used in exercising the virtual processes. Some common examples include Azure, EPANET, MATLAB, Mike urban.",
   "Realization: The methods in which the delta or difference in the parameters of the physical and digital entities is determined and either entity is updated, modified, synchronized or calibrated accordingly to ensure consistency / reconciliation. Parameters or metrics are corrected to reflect near-time data.",
   "State: The current/measured value of the parameters of either the physical or virtual entity/environment in the context of Digital Twins or variables relating to them.",
   "Metrology: Refers to measurement of a physical occurrence, including the methods, standards, and instruments used to quantify physical properties or attributes in the virtual and physical systems (how data is collected). Examples include sensors (used to record data), flow meters, or a thermometer (measuring the temperature of a physical motor) in the physical entity. A paper that talks about this would mention an occurrence that is measured and how the measurement is taken. The paper should explicitly mention something that is measured or directly refer to it.",
   "Fidelity: Degree of accuracy and completeness with which the virtual entity replicates the physical counterpart of the system, related to data, behavior, physical structure, etc. Examples include “comprehensive physical and functional description” or fully mirroring the characteristics and functionality (of physical entity).",
   "Parameters: Parameters refer to the variables, attributes, or inputs that define and characterize the behavior, performance, and state of a system within a digital twin. These parameters can be transferred from both the physical system and the virtual entity, encompassing physical properties, environmental conditions, operational settings, and performance metrics. For example, parameters may include flow rate, water quality, pressure, temperature, permeate flow, wear, flux, etc.",
   "Twinning Rate: Twinning Rate is the frequency in which the Digital Twin is updated with data from the physical system. When talking about Twinning Rate, most papers will say something about how data is updated every specified time interval or about how the Digital Twin utilizes real-time or continuous data. Any paper that mentions a certain frequency/time interval of data update or real-time Digital Twins would talk about Twinning Rate.",
   "Use-Cases: Use-Cases describe how the Digital Twin is used and in what context, for example, if the Digital Twin is described to be supplementing another process or being used to model a named facility.",
   "Perceived Benefits: The envisaged advantages of implementing a Digital Twin. Papers that talk about Perceived Benefits will mention the use of the Digital Twin to improve a system, rather than to just monitor it. Examples include cost reduction, better decision making, improved operational efficiency, proactive risk management, etc., although the context of the Digital Twin implementation should be evaluated.",
   "Data Ownership and Security: Data Ownership and Security, as stated talks about the source, ownership, OR security of the data used by the digital twin. A paper that talk about this could mention anything about data security/trust as well as sources of data.",
   "Scope: The breadth of the purview of the digital twin i.e. what portion of the physical system does the virtual entity represent/replicate.",
   "Physical-to-Virtual Connection: Means by which the state of the physical entity are transferred to the virtual environment, including IoT sensors, GIS, network gateways, scans, drone collection, field collection, citizen science, SCADA, web-services, 5G or other communication protocols",
   "Virtual-to-Physical Connection: The specific methods in which data is sent from the Virtual Part of the Digital Twin back to the physical counterpart. In other words, this facilitates the feedback mechanism in which the virtual part of the Digital Twin sends information and/or makes adjustments to the physical process. A paper that talks about Virtual-to-Physical Connection would talk about how the results from the Digital Twin would be used or how they have an effect on the physical process."
]


client2 = anthropic.Anthropic(
    api_key=claude_api_key,
)
client = openai.OpenAI(api_key=api_key)

directory = path
checked_folder_path = Path(directory) / 'Checked'
checked_folder_path.mkdir(exist_ok=True)

df_parameter_explanations = pd.DataFrame()
df_parameter_explanations2 = pd.DataFrame()

# Assuming RateLimitError is imported correctly
# Initialize counter for articles
article_count = 0

# Process each PDF file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        article_count += 1  # Increment the article counter
        file_path = Path(directory) / filename
        pdf_text = extract_text_from_pdf(file_path)
        if pdf_text:
            title = extract_title_with_gpt(client, pdf_text)
            author = extract_author_with_gpt(client, pdf_text)
            industry = extract_industry_with_gpt(client, pdf_text)
            explanation_data = {'Title': title, 'Author': author, 'Industry': industry}
            explanation_data2 = {'Title': title, 'Author': author, 'Industry': industry}

            # Initialize counter for parameters
            parameter_count = 0

            # Get explanations for each parameter if mentioned in the text
            for i, parameter in enumerate(parameters):
                definition = definitions[i]
                while True:
                    try:
                        explanation = explain_parameter_presence(client, pdf_text, parameter, definition) # Change function to explain_parameter_presence3 for Claude
                        explanation_data[parameter] = explanation
                        explanation2 = explain_parameter_presence2(client, pdf_text, parameter, definition) # Change function to explain_parameter_presence4 for Claude
                        explanation_data2[parameter] = explanation2
                        parameter_count += 1  # Increment the parameter counter
                        print(f"Article {article_count} - Parameter '{parameter}': Done")
                        break  # If successful, break out of the loop
                    except RateLimitError as e:
                        print(f"Rate limit reached for Article {article_count} - Parameter '{parameter}'. Retrying soon.")
                        client = openai.OpenAI(api_key=api_key)
                        time.sleep(1800)

            # Append explanations to the DataFrame
            df_parameter_explanations = pd.concat([df_parameter_explanations, pd.DataFrame([explanation_data])], ignore_index=True)
            df_parameter_explanations2 = pd.concat([df_parameter_explanations2, pd.DataFrame([explanation_data2])], ignore_index=True)

            # Save the DataFrame to a CSV file
            # output_explanations_csv_path = Path(directory) / 'parameter_explanations_g.csv'
            # output_explanations_csv_path2 = Path(directory) / 'parameter_explanations2_g.csv'
            # df_parameter_explanations.to_csv(output_explanations_csv_path, index=False)
            # df_parameter_explanations2.to_csv(output_explanations_csv_path, index=False)
            print(f"Article {article_count}: Done")
            #shutil.move(os.path.join(directory, filename), os.path.join(checked_folder_path, filename))

Article 1 - Parameter 'Physical Entity and Processes': Done
Article 1 - Parameter 'Virtual Entity': Done
Article 1 - Parameter 'Virtual Processes': Done
Article 1 - Parameter 'Physical Environment': Done
Article 1 - Parameter 'Virtual Environment': Done
Article 1 - Parameter 'Realization': Done
Article 1 - Parameter 'State': Done
Article 1 - Parameter 'Metrology': Done
Article 1 - Parameter 'Fidelity': Done
Article 1 - Parameter 'Parameters': Done
Article 1 - Parameter 'Twinning Rate': Done
Article 1 - Parameter 'Use Cases': Done
Article 1 - Parameter 'Perceived Benefits': Done
Article 1 - Parameter 'Data Ownership': Done
Article 1 - Parameter 'Scope': Done
Article 1 - Parameter 'P2V Connection': Done
Article 1 - Parameter 'V2P Connection': Done
Article 1: Done
Article 2 - Parameter 'Physical Entity and Processes': Done
Article 2 - Parameter 'Virtual Entity': Done
Article 2 - Parameter 'Virtual Processes': Done
Article 2 - Parameter 'Physical Environment': Done
Article 2 - Parameter 'V

In [ ]:
df_parameter_explanations.head()

,Title,Author,Industry,Physical Entity and Processes,Virtual Entity,Virtual Processes,Physical Environment,Virtual Environment,Realization,State,Metrology,Fidelity,Parameters,Twinning Rate,Use Cases,Perceived Benefits,Data Ownership,Scope,P2V Connection,V2P Connection
0,Revolutionary Technological Improvements in Am...,Corriveau et al.,General,"The text directly talks about ""Physical Entity...",The parameter 'Virtual Entity' is directly tal...,"The text does not directly talk about ""Virtual...","The text does not directly focus on the ""Physi...",The parameter 'Virtual Environment' is directl...,The concept of 'Realization' as defined in the...,The parameter 'State' is not directly mentione...,The parameter 'Metrology' is directly talked a...,The parameter 'Fidelity' is not directly menti...,The provided text does not directly focus on o...,The parameter 'Twinning Rate' is not directly ...,The parameter 'Use Cases' is not directly ment...,The parameter 'Perceived Benefits' is directly...,"The provided text does not directly focus on ""...",The parameter 'Scope' is not directly discusse...,The text does not directly talk about the 'P2V...,The provided text does not directly talk about...
1,Robotics in Construction,DeJong et al.,Construction,"Yes, the parameter 'Physical Entity and Proces...",The parameter 'Virtual Entity' is directly dis...,The parameter 'Virtual Processes' is not direc...,The parameter 'Physical Environment' is direct...,The text directly talks about the concept of a...,The provided text does not directly discuss th...,The parameter 'State' is directly talked about...,The parameter 'Metrology' is not directly talk...,The parameter 'Fidelity' is not directly menti...,"The text directly discusses the concept of ""Pa...",The text does not directly discuss the paramet...,The parameter 'Use Cases' is directly mentione...,The provided text does not directly address th...,The text does not directly focus on 'Data Owne...,The parameter 'Scope' is directly mentioned an...,The text does not directly mention or discuss ...,The text does not mention or directly talk abo...
2,Digital Twin-Based Pump Station Dynamic Schedu...,Zhou et al.,Water,The provided text directly discusses the param...,The paper directly talks about the parameter '...,The parameter 'Virtual Processes' is not expli...,The parameter 'Physical Environment' is not di...,The parameter 'Virtual Environment' is directl...,The parameter 'Realization' is not directly me...,The parameter 'State' is directly mentioned an...,The parameter 'Metrology' is not directly ment...,The parameter 'Fidelity' is not directly menti...,The paper directly talks about parameters in t...,The provided text does not directly discuss th...,The parameter 'Use Cases' is not directly ment...,The parameter 'Perceived Benefits' is directly...,The parameter 'Data Ownership' is not directly...,The parameter 'Scope' is not directly discusse...,The parameter 'P2V Connection' is directly tal...,The paper does not directly talk about the par...
3,Planning the restoration of membranes in RO de...,van Rooij et al.,Desalination,The text directly talks about the parameter 'P...,The parameter 'Virtual Entity' is directly tal...,The provided text does not directly discuss or...,"The text directly talks about the ""Physical En...",The parameter 'Virtual Environment' is directl...,"The text does not directly focus on ""Realizati...",The parameter 'State' is directly talked about...,"The paper does not directly focus on ""Metrolog...",The parameter 'Fidelity' is not directly menti...,"The text directly talks about ""parameters"" in ...",The paper does not directly focus on the param...,The parameter 'Use Cases' is not directly ment...,The parameter 'Perceived Benefits' is directly...,The text does not directly focus on 'Data Owne...,The parameter 'Scope' is not directly discusse...,"The provided text from ""Desalination 519 (2022...",The provided text does not directly discuss th...
4,Digital Water

In [ ]:
df_parameter_explanations2.head()

,Title,Author,Industry,Physical Entity and Processes,Virtual Entity,Virtual Processes,Physical Environment,Virtual Environment,Realization,State,Metrology,Fidelity,Parameters,Twinning Rate,Use Cases,Perceived Benefits,Data Ownership,Scope,P2V Connection,V2P Connection
0,Revolutionary Technological Improvements in Am...,Corriveau et al.,General,The parameter 'Physical Entity and Processes' ...,The parameter 'Virtual Entity' is indirectly r...,The parameter 'Virtual Processes' is indirectl...,The parameter 'Physical Environment' is not di...,The parameter 'Virtual Environment' is indirec...,The parameter 'Realization' is not directly me...,The parameter 'State' is not directly mentione...,The parameter 'Metrology' is not explicitly me...,The parameter 'Fidelity' is not directly menti...,The parameter 'Parameters' is not directly men...,The parameter 'Twinning Rate' is not directly ...,The parameter 'Use Cases' is not explicitly me...,The parameter 'Perceived Benefits' is not dire...,The parameter 'Data Ownership' is not directly...,The parameter 'Scope' is not directly discusse...,The parameter 'P2V Connection' is not directly...,The parameter 'V2P Connection' is not directly...
1,Robotics in Construction,DeJong et al.,Construction,The parameter 'Physical Entity and Processes' ...,The parameter 'Virtual Entity' is not mentione...,The parameter 'Virtual Processes' is not direc...,The parameter 'Physical Environment' is indire...,The parameter 'Virtual Environment' is not dir...,The parameter 'Realization' is indirectly disc...,The parameter 'State' is not directly mentione...,Metrology is not directly talked about in the ...,The parameter 'Fidelity' is not explicitly men...,The parameter 'Parameters' is not directly men...,The parameter 'Twinning Rate' is not mentioned...,The parameter 'Use Cases' is indirectly discus...,The parameter 'Perceived Benefits' is not dire...,The parameter 'Data Ownership' is not directly...,The text does mention the parameter 'Scope' in...,The parameter 'P2V Connection' is not explicit...,The parameter 'V2P Connection' is not directly...
2,Digital Twin-Based Pump Station Dynamic Schedu...,Zhou et al.,Water,The parameter 'Physical Entity and Processes' ...,The parameter 'Virtual Entity' is not directly...,The parameter 'Virtual Processes' is not direc...,The text does mention the parameter 'Physical ...,The parameter 'Virtual Environment' is indirec...,The parameter 'Realization' is indirectly talk...,The parameter 'State' is indirectly talked abo...,The parameter 'Metrology' is not mentioned or ...,The parameter 'Fidelity' is indirectly implied...,The parameter 'Parameters' is not directly men...,The parameter 'Twinning Rate' is not mentioned...,The parameter 'Use Cases' is not explicitly me...,The parameter 'Perceived Benefits' is not dire...,The parameter 'Data Ownership' is not directly...,The parameter 'Scope' is not directly talked a...,The parameter 'P2V Connection' is not directly...,The parameter 'V2P Connection' is not directly...
3,Planning the restoration of membranes in RO de...,van Rooij et al.,Desalination,The text primarily focuses on the development ...,The text does talk about the parameter 'Virtua...,The parameter 'Virtual Processes' is not expli...,The text does not directly talk about the para...,The text does not directly talk about the para...,The text directly talks about the parameter 'R...,The parameter 'State' is indirectly talked abo...,The parameter 'Metrology' is not directly talk...,The parameter 'Fidelity' is not explicitly men...,The paper focuses on the development of a deci...,The text does not directly mention the paramet...,The text focuses on the development of a decis...,The parameter 'Perceived Benefits' is not dire...,The parameter 'Data Ownership' is not mentione...,The parameter 'Scope' is not directly mentione...,The text does not directly mention the paramet...,The parameter 'V2P Connection' is not directly...
4,Digital Water Operational digital twins in the

In [ ]:
import pandas as pd
from collections import Counter

def check_parameter_mentioned(client, explanation):
    max_attempts = 3
    results = []

    for _ in range(max_attempts):
        response = client.chat.completions.create(
            model="gpt-4-0125-preview", # Change to gpt-3.5-turbo for lower cost
            messages=[
                {"role": "system", "content": "Indicate with a 1 if the explanation says the parameter is mentioned, else return 0:"},
                {"role": "user", "content": explanation}
            ],
            temperature=0,
            max_tokens=5,
            top_p=1
        )
        answer = response.choices[0].message.content.strip()

        # Check if the answer is valid ('1' or '0'), otherwise redo the response
        if answer in ['1', '0']:
            results.append(answer)
            break  # Exit loop after the first valid response

    # Calculate the most frequent result or default to 0 if no valid results
    if results:
        most_frequent_result = Counter(results).most_common(1)[0][0]
    else:
        most_frequent_result = '0'  # Default to '0' if no valid results were obtained

    return int(most_frequent_result)


# Assuming client and parameters are correctly defined and api_key is set
client = openai.OpenAI(api_key=api_key)

# Initialize two new DataFrames to store the binary values
df_parameters = pd.DataFrame(columns=df_parameter_explanations.columns)
df_parameters2 = pd.DataFrame(columns=df_parameter_explanations2.columns)

counter1, counter2 = 0, 0  # Separate counters for each DataFrame

# Iterate over each row in the first explanations DataFrame
for index, row in df_parameter_explanations.iterrows():
    binary_data = {'Title': row['Title'], 'Author': row['Author'], 'Industry': row['Industry']}
    for parameter in parameters:
        explanation = row[parameter]
        binary_data[parameter] = check_parameter_mentioned(client, explanation)
    df_parameters = pd.concat([df_parameters, pd.DataFrame([binary_data])], ignore_index=True)
    counter1 += 1
    print(f"Article {counter1} Done for DF1")

# Iterate over each row in the second explanations DataFrame
for index, row in df_parameter_explanations2.iterrows():
    binary_data = {'Title': row['Title'], 'Author': row['Author'], 'Industry': row['Industry']}
    for parameter in parameters:
        explanation = row[parameter]
        binary_data[parameter] = check_parameter_mentioned(client, explanation)
    df_parameters2 = pd.concat([df_parameters2, pd.DataFrame([binary_data])], ignore_index=True)
    counter2 += 1
    print(f"Article {counter2} Done for DF2")

Article 1 Done for DF1
Article 2 Done for DF1
Article 3 Done for DF1
Article 4 Done for DF1
Article 5 Done for DF1
Article 6 Done for DF1
Article 7 Done for DF1
Article 8 Done for DF1
Article 9 Done for DF1
Article 10 Done for DF1
Article 1 Done for DF2
Article 2 Done for DF2
Article 3 Done for DF2
Article 4 Done for DF2
Article 5 Done for DF2
Article 6 Done for DF2
Article 7 Done for DF2
Article 8 Done for DF2
Article 9 Done for DF2
Article 10 Done for DF2


In [ ]:
df_parameters.head()

,Title,Author,Industry,Physical Entity and Processes,Virtual Entity,Virtual Processes,Physical Environment,Virtual Environment,Realization,State,Metrology,Fidelity,Parameters,Twinning Rate,Use Cases,Perceived Benefits,Data Ownership,Scope,P2V Connection,V2P Connection
0,Revolutionary Technological Improvements in Am...,Corriveau et al.,General,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0
1,Robotics in Construction,DeJong et al.,Construction,1,1,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0
2,Digital Twin-Based Pump Station Dynamic Schedu...,Zhou et al.,Water,1,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0
3,Planning the restoration of membranes in RO de...,van Rooij et al.,Desalination,1,1,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0
4,Digital Water Operational digital twins in the...,Lumley et al.,Water,1,1,0,1,1,1,1,1,0,1,0,1,1,0,1,0,0


In [ ]:
df_parameters2.head()

,Title,Author,Industry,Physical Entity and Processes,Virtual Entity,Virtual Processes,Physical Environment,Virtual Environment,Realization,State,Metrology,Fidelity,Parameters,Twinning Rate,Use Cases,Perceived Benefits,Data Ownership,Scope,P2V Connection,V2P Connection
0,Revolutionary Technological Improvements in Am...,Corriveau et al.,General,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Robotics in Construction,DeJong et al.,Construction,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0
2,Digital Twin-Based Pump Station Dynamic Schedu...,Zhou et al.,Water,1,0,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0
3,Planning the restoration of membranes in RO de...,van Rooij et al.,Desalination,1,1,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0
4,Digital Water Operational digital twins in the...,Lumley et al.,Water,1,1,1,1,1,1,0,0,1,1,0,1,1,0,1,0,0


In [ ]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Define the output file name
output_file = "gpt_final_1.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # Write each DataFrame to a separate sheet
    df_parameter_explanations.to_excel(writer, sheet_name="Explanations")
    df_parameter_explanations2.to_excel(writer, sheet_name="Explanations2")
    df_parameters.to_excel(writer, sheet_name="Binary_DF1")
    df_parameters2.to_excel(writer, sheet_name="Binary_DF2")

print(f"All DataFrames saved to: {output_file}")


All DataFrames saved to: gpt_final_1.xlsx


In [ ]:
import os
from pathlib import Path
import pytesseract
from PIL import Image
import pdf2image

def ocr_from_pdf(pdf_path):
    # Convert PDF to a list of images
    images = pdf2image.convert_from_path(pdf_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image) + "\n"
    return text

directory = path
articles = []

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        file_path = Path(directory) / filename
        pdf_text = ocr_from_pdf(file_path)
        articles.append(pdf_text)

def chunk_generator(text, chunk_size):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i + chunk_size])

chunk_size = 500  # Number of words per chunk, adjust as needed
chunks = tuple()

for article in articles:
    article_chunks = tuple(chunk_generator(article, chunk_size))
    chunks = chunks + (article_chunks,)

In [ ]:
import re

# Function to clean OCR text
def clean_ocr_text(raw_text):
    # Remove erroneous newline characters and unwanted symbols
    cleaned_text = re.sub(r'\n', ' ', raw_text)  # Replace newlines with space
    cleaned_text = re.sub(r'[@“”\|]', '', cleaned_text)  # Remove specific unwanted characters
    cleaned_text = re.sub(r' – ', '-', cleaned_text)  # Correct spaced hyphens
    # Replace incorrect newlines (e.g., those within words)
    cleaned_text = re.sub(r'(?<=[a-zA-Z])-(?=[a-zA-Z])', '', cleaned_text)
    # Remove any other post-processing steps you need here
    return cleaned_text

# Use the function on the OCR result
articles = [clean_ocr_text(text) for text in articles]

The code below uses OpenAI's GPT models. To use Claude, edit lines 36 and 37 as specified.

In [ ]:
import os
from pathlib import Path
import pandas as pd
import time
from openai import RateLimitError  # Assuming this is imported correctly

# Placeholder functions for extracting text, title, author, industry, and parameter explanations
# Define or import extract_text_from_pdf, extract_title_with_gpt, extract_author_with_gpt, extract_industry_with_gpt, explain_parameter_presence3, and explain_parameter_presence4 here

df_parameter_explanationsc = pd.DataFrame()
df_parameter_explanationsc2 = pd.DataFrame()

# Assuming chunks is a tuple where each entry contains chunks of the same article
# Assuming articles is a 1d array containing the full text of each article

article_count = 0
chunk_count = 0

# Simulate processing each article's chunks
# Here, replace the loop to iterate over chunks and articles arrays
for article_chunks, full_article_text in zip(chunks, articles):
    article_count += 1
    title = extract_title_with_gpt(client, full_article_text)
    author = extract_author_with_gpt(client, full_article_text)
    industry = extract_industry_with_gpt(client, full_article_text)
    explanation_data = {'Title': title, 'Author': author, 'Industry': industry}
    explanation_data2 = {'Title': title, 'Author': author, 'Industry': industry}

    for i, parameter in enumerate(parameters):
        definition = definitions[i]
        parameter_explanations = []
        parameter_explanations2 = []

        for chunk in article_chunks:
            try:
                explanation = explain_parameter_presence(client, chunk, parameter, definition) # Change function to explain_parameter_presence3 for Claude
                explanation2 = explain_parameter_presence2(client, chunk, parameter, definition) # Change function to explain_parameter_presence4 for Claude
                parameter_explanations.append(explanation)
                parameter_explanations2.append(explanation2)
                print(f"Article {article_count} - Parameter '{parameter}' - Chunk '{(chunk_count % len(article_chunks)) + 1}': Done")
                chunk_count += 1
            except RateLimitError as e:
                print(f"Rate limit reached for Article {article_count} - Parameter '{parameter}'. Retrying soon.")
                client2 = openai.OpenAI(api_key=api_key)  # Assuming client2 needs to be reset
                time.sleep(1800)  # Wait for rate limit reset before continuing

        explanation_data[parameter] = tuple(parameter_explanations)
        explanation_data2[parameter] = tuple(parameter_explanations2)

    df_parameter_explanationsc = pd.concat([df_parameter_explanationsc, pd.DataFrame([explanation_data])], ignore_index=True)
    df_parameter_explanationsc2 = pd.concat([df_parameter_explanationsc2, pd.DataFrame([explanation_data2])], ignore_index=True)

Article 1 - Parameter 'Physical Entity and Processes' - Chunk '1': Done
Article 1 - Parameter 'Physical Entity and Processes' - Chunk '2': Done
Article 1 - Parameter 'Physical Entity and Processes' - Chunk '3': Done
Article 1 - Parameter 'Physical Entity and Processes' - Chunk '4': Done
Article 1 - Parameter 'Physical Entity and Processes' - Chunk '5': Done
Article 1 - Parameter 'Physical Entity and Processes' - Chunk '6': Done
Article 1 - Parameter 'Virtual Entity' - Chunk '1': Done
Article 1 - Parameter 'Virtual Entity' - Chunk '2': Done
Article 1 - Parameter 'Virtual Entity' - Chunk '3': Done
Article 1 - Parameter 'Virtual Entity' - Chunk '4': Done
Article 1 - Parameter 'Virtual Entity' - Chunk '5': Done
Article 1 - Parameter 'Virtual Entity' - Chunk '6': Done
Article 1 - Parameter 'Virtual Processes' - Chunk '1': Done
Article 1 - Parameter 'Virtual Processes' - Chunk '2': Done
Article 1 - Parameter 'Virtual Processes' - Chunk '3': Done
Article 1 - Parameter 'Virtual Processes' - Ch

In [ ]:
import pandas as pd

def check_parameter_mentioned_in_multiple_chunks(client, explanations):
    """
    Checks if a parameter is mentioned in at least two chunks of an article.
    If the parameter is mentioned in two or more chunks, returns 1, otherwise returns 0.
    """
    mention_count = 0
    for explanation in explanations:
        max_attempts = 3
        for attempt in range(max_attempts):
            response = client.chat.completions.create(
                model="gpt-4-0125-preview",
                messages=[
                    {"role": "system", "content": "Indicate with a 1 if the explanation says the parameter is mentioned, else return 0:"},
                    {"role": "user", "content": explanation}
                ],
                temperature=0,
                max_tokens=5,
                top_p=1
            )
            answer = response.choices[0].message.content.strip()

            if answer == '1':
                mention_count += 1
                break  # No need to re-attempt for this chunk, move to the next
            elif answer == '0':
                break  # Chunk does not mention the parameter, move to the next

        if mention_count >= 1:
            return 1  # Parameter is mentioned in two or more chunks

    return 0  # Parameter is not mentioned in at least two chunks


# Initialize DataFrame to store binary values indicating parameter mentions
df_parameters_binary = pd.DataFrame(columns=['Title', 'Author', 'Industry'] + parameters)
df_parameters_binary2 = pd.DataFrame(columns=['Title', 'Author', 'Industry'] + parameters)

# Assuming 'df_parameter_explanationsc' contains the tuples with explanations for each chunk
for index, row in df_parameter_explanationsc.iterrows():
    binary_data = {'Title': row['Title'], 'Author': row['Author'], 'Industry': row['Industry']}

    for parameter in parameters:
        explanations = row[parameter]  # This is a tuple with explanations for each chunk
        binary_data[parameter] = check_parameter_mentioned_in_multiple_chunks(client, explanations)

    df_parameters_binary = pd.concat([df_parameters_binary, pd.DataFrame([binary_data])], ignore_index=True)

for index, row in df_parameter_explanationsc2.iterrows():
    binary_data2 = {'Title': row['Title'], 'Author': row['Author'], 'Industry': row['Industry']}

    for parameter in parameters:
        explanations2 = row[parameter]  # This is a tuple with explanations for each chunk
        binary_data2[parameter] = check_parameter_mentioned_in_multiple_chunks(client, explanations)

    df_parameters_binary2 = pd.concat([df_parameters_binary2, pd.DataFrame([binary_data2])], ignore_index=True)

In [ ]:
df_parameters_binary.head()

,Title,Author,Industry,Physical Entity and Processes,Virtual Entity,Virtual Processes,Physical Environment,Virtual Environment,Realization,State,Metrology,Fidelity,Parameters,Twinning Rate,Use Cases,Perceived Benefits,Data Ownership,Scope,P2V Connection,V2P Connection
0,Embrace Digital Tools to Enhance Water Utility...,Corriveau et al.,Water,1,1,1,0,1,0,1,1,0,0,1,0,1,0,0,1,0
1,International Conference on Smart Infrastructu...,DeJong et al.,Construction,1,1,0,1,1,0,1,1,0,0,1,1,1,1,1,1,0
2,Water Resources Management,Zhou et al.,Water,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1
3,Planning the restoration of membranes in RO de...,van Rooij et al.,Desalination,1,1,1,1,1,0,1,1,0,1,0,0,1,0,1,0,0
4,Operational digital twins in the urban water s...,ValverdePérez et al.,Water,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0


In [ ]:
df_parameters_binary2.head()

,Title,Author,Industry,Physical Entity and Processes,Virtual Entity,Virtual Processes,Physical Environment,Virtual Environment,Realization,State,Metrology,Fidelity,Parameters,Twinning Rate,Use Cases,Perceived Benefits,Data Ownership,Scope,P2V Connection,V2P Connection
0,Embrace Digital Tools to Enhance Water Utility...,Corriveau et al.,Water,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,International Conference on Smart Infrastructu...,DeJong et al.,Construction,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,Water Resources Management,Zhou et al.,Water,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,Planning the restoration of membranes in RO de...,van Rooij et al.,Desalination,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,Operational digital twins in the urban water s...,ValverdePérez et al.,Water,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
!pip install xlsxwriter

In [ ]:
import pandas as pd

# Define the output file name
output_file = "gpt_final_chunks_1.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # Write each DataFrame to a separate sheet
    df_parameters_binary.to_excel(writer, sheet_name="Binary_DF1")
    df_parameters_binary2.to_excel(writer, sheet_name="Binary_DF2")
    df_parameter_explanationsc.to_excel(writer, sheet_name="Explanations")
    df_parameter_explanationsc2.to_excel(writer, sheet_name="Explanations2")

print(f"All DataFrames saved to: {output_file}")

All DataFrames saved to: gpt_final_chunks_1.xlsx
